In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

def score_ohe(data,feature,ohe):
    df = data.copy()
    for f in feature:
        df.rename(columns={f'{f}':f'{f}_{ohe}'}, inplace=True)
    return df

def make_rank(data, feature):
    df = data.copy()
    for f in feature:
        df[f"{f}_rank"] = df[f].apply(lambda x: sorted(df[f].values, reverse=True).index(x)+1)
    return df

def make_rank_bin(data, feature):
    df = data.copy()
    def make_bin(x):
        if x<=4: return 1
        elif x<=11: return 2
        elif x<=23: return 3
        elif x<=40: return 4
        elif x<=60: return 5
        elif x<=77: return 6
        elif x<=89: return 7
        elif x<=96: return 8
        else: return 9
    
    for f in feature:
        df[f"{f}_bin"] = df[f].apply(lambda x: x/len(df)*100)
        
    for f in feature:
        df[f"{f}_bin"] = df[f"{f}_bin"].apply(lambda x: make_bin(x))
    return df

def make_scaling(data,feature):
    df = data.copy()  
    scaler = StandardScaler()
    
    for f in feature:
        df[f"{f[:2]}_scale"] = scaler.fit_transform(df[[f]]).reshape(-1,1)
    
    return df

In [13]:
### 데이터 불러오기
math = pd.read_csv("../../1.raw_data/student-mat.csv")
port = pd.read_csv("../../1.raw_data/student-por.csv")

### 과목 피쳐 추가
math["subject"]="math"
port["subject"]="port"

### Both 테이블 생성
both = pd.concat([math,port], axis=0)

### Both GT 생성
both["GT_scale"]=both["G1_scale"]+both["G2_scale"]+both["G3_scale"]
both = make_scaling(both, ["GT_scale"])

### 등수 추가
both = make_rank(both, ["G1_scale","G2_scale","G3_scale","GT_scale"])

### 등급 추가
both = make_rank_bin(both, ["G1_scale_rank","G2_scale_rank","G3_scale_rank","GT_scale_rank"])

### 음주량 총합
both["Talc"] = both["Walc"]+both["Dalc"]

### 저장
both.to_csv("both.csv", index=False)